# **Pairwise Sequence Alignment** 

---



We will write the code of Smith Waterman Algorithm (for local alignment) and Needleman Wunsch Algorithm (for global alignment).

We start with the Smith Waterman Algorithm since it's traceback is simpler.

In [4]:
sw=function (s1,s2,match,mismatch,gap)
{
  stopifnot(gap <= 0) #user check
  stopifnot(mismatch <= 0) #user check
  stopifnot(match >= 0) # user check
  l1 = nchar(s1)
  l2 = nchar(s2)
  s1 = unlist(strsplit(s1, split = "")) 
  s2 = unlist(strsplit(s2, split = ""))
  F1 = matrix(0, nrow = l1 + 1, ncol = l2 + 1)
  rownames(F1) = c(" ", s1)
  colnames(F1) = c(" ", s2)
  F1[1, ] = cumsum(rep(0,l2+1)) #1st row fill
  F1[, 1] = cumsum( rep(0,l1+1)) # 1st col fill
  D = matrix(list(), nrow = l1 + 1, ncol = l2 + 1)
  rownames(D) = c(" ", s1)
  colnames(D) = c(" ", s2)
  D[1, ] = rep("horiz")
  D[, 1] = rep("vertic")
  type = c("diag", "horiz", "vertic")
  for (i in 2:(l1 + 1)) {
    for (j in 2:(l2 + 1)) {
      horiz = F1[i, j - 1] + gap
      vertic = F1[i - 1, j] + gap
      diag = ifelse(rownames(F1)[i] == colnames(F1)[j], 
                    F1[i -1, j - 1] + match, F1[i - 1, j - 1] + mismatch)
      entry=c(diag, horiz, vertic)
      F1[i, j] = max(0,diag, horiz, vertic) #entries of F matrix (F means False value in R hence I used the symbol F1)
      D[[i, j]] = type[which(entry == max(entry))] #entries of direction matrix
    }
  }
  #optimal sequences:
  df = data.frame()
  s = NULL
  for (i in 1:(l1+1)) {
    for (j in 1:(l2+1)) {
      if(F1[i,j] == max(F1)) df = rbind(c(i,j,"",""),df)#get the maximum value
    }
  } #trace all possible alignments
  while (dim(df)[1]>0) {
    path = df[1,]
    df = df[-1,]
    if(as.numeric(path[1])==1 && as.numeric(path[2])==1){
      s=rbind(s,c(path[3],path[4]))
    }
    else{
      initz = D[[as.numeric(path[1]),as.numeric(path[2])]]
      if("diag" %in% initz){
        df = rbind(c(as.numeric(path[1])-1,as.numeric(path[2])-1,
                        paste(rownames(F1)[as.numeric(path[1])],path[3],sep=""),
                        paste(colnames(F1)[as.numeric(path[2])],path[4],sep="")),
                      df)
      }
      if("horiz" %in% initz){
        df = rbind(c(as.numeric(path[1]),as.numeric(path[2])-1,
                        paste("-",path[3],sep=""),paste(colnames(F1)
                                                        [as.numeric(path[2])],path[4],sep="")),df)
      }
      if("vertic" %in% initz){
        df = rbind(c(as.numeric(path[1])-1,as.numeric(path[2]),
                        paste(rownames(F1)[as.numeric(path[1])],path[3],sep=""),
                        paste("-",path[4],sep="")),df)
      }
    }
  }
  colnames(s)<-c("S1","S2")
  return(list(aligned_seqs = s,score = max(F1),
              score_matrix = F1, direction_matrix = D))
}



Code is complete, now let us take two pais of simple sequences to verify it's correctness.I deliberately take the sequences such that it is quite intuitve to guess the alignments.

In [5]:
sw("ABCDABC","ABC",2,-1,-5)$aligned_seqs

S1      S2     
[1,] ABCDABC ----ABC
[2,] ABC     ABC

In [7]:
sw("ATGCATGCATGC","ATCTGC",1,-1,-1)$aligned_seqs

S1           S2          
[1,] ATGCATGCATGC ----AT-C-TGC
[2,] ATGCATGC     AT-C-TGC

We proceed similarly, hence not writing all the comments, only changes are in the extra 0 of smith waterman which is not here and the traceback starts differently.

In [8]:
nw=function (s1,s2,match,mismatch,gap)
{
  stopifnot(gap <= 0) 
  stopifnot(mismatch <= 0) 
  stopifnot(match >= 0) 
  l1 = nchar(s1)
  l2 = nchar(s2)
  s1 = unlist(strsplit(s1, split = "")) 
  s2 = unlist(strsplit(s2, split = ""))
  F1 = matrix(0, nrow = l1 + 1, ncol = l2 + 1)
  rownames(F1) = c(" ", s1)
  colnames(F1) = c(" ", s2)
  F1[1, ] = cumsum(c(0, rep(gap, l2))) 
  F1[, 1] = cumsum(c(0, rep(gap, l1))) 
  D = matrix(list(), l1 + 1, l2 + 1)
  rownames(D) = c(" ", s1)
  colnames(D) = c(" ", s2)
  D[1, ] = rep("horiz")
  D[, 1] = rep("vertic")
  type = c("diag", "horiz", "vertic")
  for (i in 2:(l1 + 1)) {
    for (j in 2:(l2 + 1)) {
      horiz = F1[i, j - 1] + gap
      vertic = F1[i - 1, j] + gap
      diag = ifelse(rownames(F1)[i] == colnames(F1)[j], 
                    F1[i -1, j - 1] + match, F1[i - 1, j - 1] + mismatch)
      entry=c(diag, horiz, vertic)
      F1[i, j] = max(diag, horiz, vertic) #no zero
      D[[i, j]] = type[which(entry == max(entry))]
    }
  }

  df = data.frame()
  s = NULL #traceback from last element not max
  df = rbind(c(l1+1,l2+1,"",""),df)
  while (dim(df)[1]>0) {
    path = df[1,]
    df = df[-1,]
    if(as.numeric(path[1])==1 && as.numeric(path[2])==1){
      s=rbind(s,c(path[3],path[4]))
    }
    else{
      initz = D[[as.numeric(path[1]),as.numeric(path[2])]]
      if("diag" %in% initz){
        df = rbind(c(as.numeric(path[1])-1,as.numeric(path[2])-1,
                        paste(rownames(F1)[as.numeric(path[1])],path[3],sep=""),
                        paste(colnames(F1)[as.numeric(path[2])],path[4],sep="")),
                      df)
      }
      if("horiz" %in% initz){
        df = rbind(c(as.numeric(path[1]),as.numeric(path[2])-1,
                        paste("-",path[3],sep=""),paste(colnames(F1)
                                                        [as.numeric(path[2])],path[4],sep="")),df)
      }
      if("vertic" %in% initz){
        df = rbind(c(as.numeric(path[1])-1,as.numeric(path[2]),
                        paste(rownames(F1)[as.numeric(path[1])],path[3],sep=""),
                        paste("-",path[4],sep="")),df)
      }
    }
  }
  colnames(s)<-c("S1","S2")
  return(list(aligned_seqs = s,score = F1[nrow(F1), ncol(F1)],
              score_matrix = F1, movement_matrix = D))
}

In [9]:
nw("FMDTPLNE","FKHMEDPLE",1,-2,-2)$aligned_seqs

S1,S2
<chr>,<chr>
F--M-DTPLNE,FKHMED-PL-E


Oops! just one optimal alignment, take two other choices.

In [12]:
nw("ATGCATGCATGC","ATCTGC",1,-1,-1)$aligned_seqs

S1           S2          
[1,] ATGCATGCATGC AT-C-TGC----
[2,] ATGCATGCATGC AT-C-TG----C
[3,] ATGCATGCATGC AT-C-T----GC
[4,] ATGCATGCATGC AT-C-----TGC
[5,] ATGCATGCATGC AT-----C-TGC
[6,] ATGCATGCATGC A----T-C-TGC
[7,] ATGCATGCATGC ----AT-C-TGC

**I have just printed the alignments, ingeneral the function returns the score matrix, direction matrix too, one can run it and see if interested**